In [1]:
from javascript import require, On, Once, AsyncTask, once, off
vec3 = require("vec3")
import math

AggregateError [ECONNREFUSED]: 

    at internalConnectMultiple (node:net:1117:18)

    at afterConnectMultiple (node:net:1684:7) {

  code: 'ECONNREFUSED',

  [errors]: [

    Error: connect ECONNREFUSED ::1:3000

        at createConnectionError (node:net:1647:14)

        at afterConnectMultiple (node:net:1677:16) {

      errno: -4078,

      code: 'ECONNREFUSED',

      syscall: 'connect',

      address: '::1',

      port: 3000

    },

    Error: connect ECONNREFUSED 127.0.0.1:3000

        at createConnectionError (node:net:1647:14)

        at afterConnectMultiple (node:net:1677:16) {

      errno: -4078,

      code: 'ECONNREFUSED',

      syscall: 'connect',

      address: '127.0.0.1',

      port: 3000

    }

  ]

}

AggregateError [ECONNREFUSED]: 

    at internalConnectMultiple (node:net:1117:18)

    at afterConnectMultiple (node:net:1684:7) {

  code: 'ECONNREFUSED',

  [errors]: [

    Error: connect ECONNREFUSED ::1:3000

        at createConnectionError (node:net:

In [2]:
# Import the javascript libraries
mineflayer = require("mineflayer")

# Global bot parameters
serverHost = "localhost"
serverPort = 3000
reconnect = True

In [3]:

class Agent:
    def __init__(self, botName, serverHost, serverPort, reconnect=True):
        self.botName = botName
        self.serverHost = serverHost
        self.serverPort = serverPort
        self.reconnect = reconnect
        self.bot = None
        self.botArgs = {
            "host": serverHost,
            "port": serverPort,
            "username": botName,
            "hideErrors": False,
        }
        self.createBot()

    def createBot(self):
        self.bot = mineflayer.createBot(self.botArgs)
        self.startEvents()
        return self.bot
    
    def log(self, message):
        print(f"[{self.bot.username}] {message}")

    def startEvents(self):
        bot = self.bot
        botName = self.botName
        reconnectState = {"reconnect": self.reconnect}

        @On(bot, "login")
        def handleLogin(this):
            botSocket = bot._client.socket
            print(f"[{botName}] Logged in to {botSocket.server if botSocket.server else botSocket._host }")

        @On(bot, "kicked")
        def handleKicked(this, reason, loggedIn):
            if loggedIn:
                print(f"[{botName}] Kicked whilst trying to connect: {reason}")

        @On(bot, "messagestr")
        def handleMessagestr(this, message, messagePosition, jsonMsg, sender, verified=None):
            if messagePosition == "chat" and "quit" in message:
                reconnectState["reconnect"] = False
                this.quit()

        @On(bot, "end")
        def handleEnd(this, reason):
            print(f"[{botName}] Disconnected: {reason}")
            # Turn off old events
            off(bot, "login", handleLogin)
            off(bot, "kicked", handleKicked)
            off(bot, "messagestr", handleMessagestr)

            # Reconnect if the reconnect flag is set to True
            if reconnectState["reconnect"]:
                print(f"[{botName}] Attempting to reconnect")
                self.createBot()

            # Last event listener
            off(bot, "end", handleEnd)

    def setControlState(self, control, state):
        if self.bot:
            self.bot.setControlState(control, state)
        else:
            print("Bot is not created yet. Call createBot() first.")

    def move(self, direction, blocks):
        if direction in ["forward", "back", "left", "right"]:
            startPosition = self.bot.entity.position.clone()

            def calculateDistance(pos1, pos2):
                return math.sqrt((pos1.x - pos2.x) ** 2 + (pos1.z - pos2.z) ** 2)

            def checkDistance(*args):
                currentPosition = self.bot.entity.position
                distance = calculateDistance(startPosition, currentPosition)
                if distance >= blocks:
                    self.bot.setControlState(direction, False)
                else:
                    self.bot.once('move', checkDistance)

            self.setControlState(direction, True)
            self.bot.once('move', checkDistance)
        else:
            print(f"Invalid move direction: {direction}")

    def forward(self, blocks):
        self.move("forward", blocks)

    def back(self, blocks):
        self.move("back", blocks)

    def left(self, blocks):
        self.move("left", blocks)

    def right(self, blocks):
        self.move("right", blocks)

    def jump(self, state=True):
        self.setControlState("jump", state)

    def stop(self):
        # Stops all movement
        self.setControlState("forward", False)
        self.setControlState("back", False)
        self.setControlState("left", False)
        self.setControlState("right", False)
        self.setControlState("jump", False)




In [4]:
agent = Agent("Johnny", "localhost", 3000)
agent1 = Agent("John", "localhost", 3000)
agent2 = Agent("Jon", "localhost", 3000)


[Johnny] Logged in to localhost
[John] Logged in to localhost
[Jon] Logged in to localhost


In [7]:
agent1.forward(2)

what is happening
what is happening
what is happening
what is happening
what is happening
what is happening
what is happening
what is happening
[Jon] Disconnected: socketClosed
[Jon] Attempting to reconnect
[Johnny] Disconnected: socketClosed
[Johnny] Attempting to reconnect
[John] Disconnected: socketClosed
[John] Attempting to reconnect
